# 2024 OIBC Challenge
## (Deep Learning version skeleton 01)

In [3]:
import torch

In [4]:
if torch.cuda.is_available():
  print(torch.cuda.get_device_name())
  print(torch.__version__)
  print(torch.version.cuda)
  x = torch.randn(1).cuda()
  print(x)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

NVIDIA GeForce GTX 1650
2.5.0+cu118
11.8
tensor([-1.2571], device='cuda:0')


device(type='cuda', index=0)

In [ ]:
data